In [ ]:
import osm2geojson
import geopandas as gpd
import networkx as nx
import osmnx as ox
import momepy
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython import display
from shapely.ops import nearest_points,substring
from shapely import from_wkt,Point,wkt,LineString,geometry
from fiona.errors import DriverError
from itertools import chain
import requests
import time
from json import JSONDecodeError
tqdm.pandas()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('mode.chained_assignment', None)

def get_boundary(osm_id):

    overpass_url = "http://lz4.overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
            (
              relation({osm_id});
            );
    out geom;
    """
    result = requests.get(overpass_url, params={'data': overpass_query})
    json_result = result.json()

    return json_result


def get_routes(osm_id, public_transport_type):

    overpass_url = "http://lz4.overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
            (
                relation({osm_id});
            );map_to_area;
            (
                relation(area)['route'='{public_transport_type}'];
            );
    out geom;
    """
    result = requests.get(overpass_url, params={'data': overpass_query})
    json_result = result.json()["elements"]

    return pd.DataFrame(json_result)


def overpass_query(func, *args, attempts=5):

    for i in range(attempts):
        try:
            return func(*args)
        except JSONDecodeError:
            print("Another attempt to get response from Overpass API...")
            time.sleep(20)
            continue

    raise SystemError(
    """Something went wrong with Overpass API when JSON was parsed. Check the query and to send it later.""")


def parse_overpass_route_response(loc, city_crs, boundary):

    route = pd.DataFrame(loc['members'])
    ways = route[route['type'] == 'way']
    if len(ways) > 0:
        ways = ways['geometry'].reset_index(drop = True)
        ways = ways.apply(lambda x: pd.DataFrame(x))
        ways = ways.apply(lambda x: LineString(list(zip(x['lon'], x['lat']))))
        ways = gpd.GeoDataFrame(ways.rename("geometry")).set_crs(4326)
        if ways.within(boundary).all():
            # fix topological errors and then make LineString from MultiLineString
            ways = get_linestring(ways.to_crs(city_crs))
        else:
            ways = None
    else:
        ways = None

    if "node" in route["type"].unique():
        platforms = route[route['type'] == 'node'][["lat", "lon"]].reset_index(drop = True)
        platforms = platforms.apply(lambda x: Point(x["lon"], x["lat"]), axis=1)
    else:
        platforms = None

    return pd.Series({"way": ways, "platforms": platforms})


def get_linestring(route):

    equal_lines = route.apply(lambda x: find_equals_line(x, route), axis=1).dropna()
    lines_todel = list(chain(*[line[1:] for line in list(equal_lines)]))
    route = route.drop(lines_todel).reset_index()

    path_buff = gpd.GeoSeries(route.geometry.buffer(0.01))
    connect_series = route.apply(lambda x: find_connection(x, path_buff), axis=1).dropna()
    sequences = get_sequences(connect_series, [])
    if sequences is None:
        return None

    len_sequence = [len(sec) for sec in sequences]
    max_sequence = len_sequence.index(max(len_sequence))
    sequence = sequences[max_sequence]
    comlete_line = [route.geometry[sequence[0]]]

    for i in range(len(sequence) - 1):
        line1 = comlete_line[i]
        line2 = route.geometry[sequence[i + 1]]
        con_point1, con_point2 = nearest_points(line1, line2)

        line2 = list(line2.coords)
        check_reverse = gpd.GeoSeries([Point(line2[0]), Point(line2[-1])]).distance(con_point2).idxmin()
        if check_reverse == 1:
            line2.reverse()

        comlete_line.append(LineString(line2))

    comlete_line = list(chain(*[list(line.coords) for line in comlete_line]))
    comlete_line = list(pd.Series(comlete_line).drop_duplicates())

    return LineString(comlete_line)


def find_equals_line(loc, series):

    series = series.drop(loc.name)
    eq_lines = series.geometry.apply(lambda x: x.almost_equals(loc.geometry))
    eq_lines = series[eq_lines].index

    equal_lines = sorted(list(eq_lines) + [loc.name]) if len(eq_lines) > 0 else None

    return equal_lines


def find_connection(loc, df):

    df = df.drop(loc.name)
    bool_ser = df.intersects(loc.geometry)
    connect_lines = df[bool_ser].index

    if len(connect_lines) > 0:
        return list(connect_lines)
    else:
        return None


def get_sequences(connect_ser, sequences=[]):

    num_con = connect_ser.apply(lambda x: len(x))
    finite_points = pd.DataFrame(connect_ser[num_con == 1].rename("value"))

    if len(finite_points) == 0:
        return None

    sequence = move_next(finite_points.index[0], connect_ser, [])
    sequences.append(sequence)

    route_finite = finite_points.index.isin(sequence)
    if route_finite.all():
        return sequences
    else:
        connect_ser = connect_ser.drop(finite_points.index[route_finite])
        sequences = get_sequences(connect_ser, sequences)
        return sequences

def move_next(path, series, sequence, branching=None):

    sequence.append(path)
    try:
        series = series.drop(path)
    except: pass
    bool_next_path = series.apply(lambda x: path in x)
    next_path = series[bool_next_path].index

    if len(next_path) == 0:
        return sequence

    elif len(next_path) > 1:

        if branching is None:
            branches_start = path
            sequence_variance = []
            for path in next_path:
                series_ = series.drop([path_ for path_ in next_path if path_ != path])
                sequence_ = move_next(path, series_, [], branches_start)
                sequence_variance.append(sequence_)

        else:
            return sequence

        len_sequence = [len(sec) for sec in sequence_variance]
        max_sequence = len_sequence.index(max(len_sequence))
        sequence_ = sequence_variance[max_sequence]
        series_ = series.drop(list(chain(*[sec[-2:] for sec in sequence_variance])))
        sequence = sequence + sequence_
        sequence_ = move_next(sequence_[-1], series_, sequence_, None)
        return sequence + sequence_

    else:
        sequence = move_next(next_path[0], series, sequence, branching)
    return sequence



"""

Function project_platforms and its supplementary functions are used to project points on lines.
It is a necessary operation since OpenStreetMap contains two types of points describing
public transport stops - 'stop' and 'platforms'. The points marked as 'platform' usually
do not lie on route lines. Moreover some of them are very close to each other and probably
mean the same stop. To check this, 'project_threshold' value and recursion function are used.

Function project_platforms takes two arguments - 'loc' which is Series contains rows 'platforms' and 'way'
and 'city_crs'. 'way' is shapely LineString object, and 'platforms' is Series of shapely Point objects

loc: Series object
city_crs: int


"""

def project_platforms(loc, city_crs):

    project_threshold = 5
    edge_indent = 10

    platforms = loc["platforms"]
    line = loc["way"]
    line_length = line.length

    if platforms is not None:
        platforms = gpd.GeoSeries(platforms).set_crs(4326).to_crs(city_crs)
        stops = platforms.apply(lambda x: nearest_points(line, x)[0])
        stops = gpd.GeoDataFrame(stops).rename(columns={0:"geometry"}).set_geometry("geometry")
        stops = recursion(stops, project_threshold)

        check_reverse = gpd.GeoSeries([Point(line.coords[0]), Point(line.coords[-1])]).distance(stops[0]).idxmin()
        if check_reverse == 1:
            line = list(line.coords)
            line.reverse()
            line = LineString(line)

        stops_distance = stops.apply(lambda x: line.project(x)).sort_values()
        stops = stops.loc[stops_distance.index]
        condition = (stops_distance > edge_indent)&(stops_distance < line_length - edge_indent)
        stops, distance = stops[condition].reset_index(drop=True), [0] + list(stops_distance[condition])
        distance.append(line_length)

        if len(stops) > 0:
            start_line = gpd.GeoSeries(Point(line.coords[0])).set_crs(city_crs)
            end_line = gpd.GeoSeries(Point(line.coords[-1])).set_crs(city_crs)
            stops = pd.concat([start_line, stops, end_line]).reset_index(drop=True)
        else:
            stops, distance = get_line_from_start_to_end(line, line_length)
    else:
        stops, distance = get_line_from_start_to_end(line, line_length)

    pathes = [[tuple(round(c, 2) for c in stops[i].coords[0]),
               tuple(round(c, 2) for c in stops[i + 1].coords[0])]
               for i in range(len(stops) - 1)]

    return pd.Series({"pathes": pathes, "distance": distance})


def recursion (stops, threshold):

    stops['to_del'] = stops.apply(lambda x: get_index_to_delete(stops, x, threshold), axis = 1)

    if stops['to_del'].isna().all():
        return stops["geometry"]
    else:
        stops_near_pair = stops.dropna().apply(lambda x: sorted([x.name, x.to_del]), axis=1)
        stops_to_del = [pair[0] for pair in stops_near_pair]
        stops = stops.drop(stops_to_del)
        stops = recursion(stops, threshold)

    return stops.reset_index(drop=True)


def get_index_to_delete(other_stops, loc_stop, threshold):

    stops_to_del = other_stops.geometry.distance(loc_stop.geometry).sort_values().drop(loc_stop.name)
    stops_to_del = list(stops_to_del[stops_to_del < threshold].index)

    if len(stops_to_del) > 0:
        return stops_to_del[0]
    else:
        return None


def get_line_from_start_to_end(line, line_length):

    start_line = gpd.GeoSeries(Point(line.coords[0]))
    end_line = gpd.GeoSeries(Point(line.coords[-1]))
    stops = pd.concat([start_line, end_line]).reset_index(drop=True)
    distance = [0, line_length]

    return stops, distance

"""

These bunch of functions are used to make spatial union of two graphs.

"""

def get_nearest_edge_geometry(points, G):

    G = G.edge_subgraph([(u, v, n) for u, v, n, e in G.edges(data=True, keys=True) if e["type"] == "walk"])
    G = convert_geometry(G.copy())
    coords = list(points.geometry.apply(lambda x: list(x.coords)[0]))
    x = [c[0] for c in list(coords)]
    y = [c[1] for c in list(coords)]
    edges, distance = ox.distance.nearest_edges(G, x, y, return_dist=True)
    edges_geom = list(map(lambda x: (x, G[x[0]][x[1]][x[2]]["geometry"]), edges))
    edges_geom = pd.DataFrame(edges_geom, index=points.index, columns=["edge_id", "edge_geometry"])
    edges_geom["distance_to_edge"] = distance
    return pd.concat([points, edges_geom], axis=1)

def convert_geometry(graph):
    for u, v, n, data in graph.edges(data=True, keys=True):
        data["geometry"] = wkt.loads(data["geometry"])
    return graph

def project_point_on_edge(points_edge_geom):

    points_edge_geom["nearest_point_geometry"] = points_edge_geom.apply(
        lambda x: nearest_points(x.edge_geometry, x.geometry)[0], axis=1)
    points_edge_geom["len"] = points_edge_geom.apply(
        lambda x: x.edge_geometry.length, axis=1)
    points_edge_geom["len_from_start"] = points_edge_geom.apply(
        lambda x: x.edge_geometry.project(x.geometry) , axis=1)
    points_edge_geom["len_to_end"] = points_edge_geom.apply(
        lambda x: x.edge_geometry.length - x.len_from_start, axis=1)

    return points_edge_geom


def update_edges(points_info, G):

    G_with_drop_edges = delete_edges(points_info, G)
    updated_G, split_points = add_splitted_edges(G_with_drop_edges, points_info)
    updated_G, split_points = add_connecting_edges(updated_G, split_points)

    return updated_G, split_points


def delete_edges(project_points, G):

    bunch_edges = []
    G_copy = convert_geometry(G.copy())
    for e in list(project_points["edge_id"]):
        flag = check_parallel_edge(G_copy, *e)
        if flag == 2:
            bunch_edges.extend([(e[0], e[1], e[2]), (e[1], e[0], e[2])])
        else:
            bunch_edges.append((e[0], e[1], e[2]))

    bunch_edges = list(set(bunch_edges))
    G.remove_edges_from(bunch_edges)

    return G


def check_parallel_edge(G, u, v, n):

    if u == v:
        return 1
    elif G.has_edge(u, v) and G.has_edge(v, u):
        if G[u][v][n]["geometry"].equals(G[v][u][n]["geometry"]):
            return 2
        else:
            return 1
    else:
        return 1


def add_splitted_edges(G, split_nodes):

    start_node_idx = max((G.nodes)) + 1
    split_nodes["node_id"] = range(start_node_idx, start_node_idx + len(split_nodes))
    nodes_bunch = split_nodes.apply(lambda x: generate_nodes_bunch(x), axis=1)
    nodes_attr = split_nodes.set_index("node_id").nearest_point_geometry.apply(
        lambda x: {"x": round(list(x.coords)[0][0], 2), "y": round(list(x.coords)[0][1], 2)}).to_dict()
    G.add_edges_from(list(nodes_bunch.explode()))
    nx.set_node_attributes(G, nodes_attr)

    return G, split_nodes


def generate_nodes_bunch(split_point):

    edge_pair = []
    edge_nodes = split_point.edge_id
    edge_geom_ = split_point.edge_geometry
    new_node_id = split_point.node_id
    len_from_start = split_point.len_from_start
    len_to_end = split_point.len_to_end
    len_edge = split_point.len

    fst_edge_attr = {
        'length_meter': len_from_start, "geometry": str(substring(edge_geom_, 0, len_from_start)), "type": "walk",
        }
    snd_edge_attr = {
        'length_meter': len_to_end, "geometry": str(substring(edge_geom_, len_from_start, len_edge)), "type": "walk",
        }
    edge_pair.extend([(edge_nodes[0], new_node_id, fst_edge_attr),(new_node_id, edge_nodes[0], fst_edge_attr),
                      (new_node_id, edge_nodes[1], snd_edge_attr), (edge_nodes[1], new_node_id, snd_edge_attr)])

    return edge_pair


def add_connecting_edges(G, split_nodes):

    start_node_idx = split_nodes["node_id"].max() + 1
    split_nodes["connecting_node_id"] = list(range(start_node_idx, start_node_idx + len(split_nodes)))
    nodes_attr = split_nodes.set_index("connecting_node_id").geometry.apply(
        lambda p: {"x": round(p.coords[0][0], 2), "y": round(p.coords[0][1], 2)}
        ).to_dict()
    conn_edges = split_nodes.apply(
        lambda x: (x.node_id, x.connecting_node_id, {
            "type": "walk", "length_meter": round(x.distance_to_edge, 3),
            "geometry": str(LineString([x.geometry, x.nearest_point_geometry]))
            }), axis=1)
    conn_edges_another_direct = conn_edges.apply(lambda x: (x[1], x[0], x[2]))
    G.add_edges_from(conn_edges.tolist() + conn_edges_another_direct.tolist())
    nx.set_node_attributes(G, nodes_attr)
    return G, split_nodes


def join_graph(G_base, G_to_project, points_df):

    new_nodes = points_df.set_index("node_id_to_project")["connecting_node_id"]
    for n1, n2, d in tqdm(G_to_project.edges(data=True)):
        G_base.add_edge(int(new_nodes[n1]), int(new_nodes[n2]), **d)
        nx.set_node_attributes(
            G_base, {int(new_nodes[n1]): G_to_project.nodes[n1], int(new_nodes[n2]): G_to_project.nodes[n2]}
            )

    return G_base

def get_osmnx_graph(city_osm_id, city_crs, graph_type, speed=None):

    boundary = overpass_query(get_boundary, city_osm_id)
    boundary = osm2geojson.json2geojson(boundary)
    boundary = gpd.GeoDataFrame.from_features(boundary["features"]).set_crs(4326)

    print(f"Extracting and preparing {graph_type} graph...")
    G_ox = ox.graph.graph_from_polygon(polygon=boundary["geometry"][0], network_type=graph_type)
    G_ox.graph["approach"] = "primal"

    nodes, edges = momepy.nx_to_gdf(G_ox, points=True, lines=True, spatial_weights=False)
    nodes = nodes.to_crs(city_crs).set_index("nodeID")
    nodes_coord = nodes.geometry.apply(
        lambda p: {"x": round(p.coords[0][0], 2), "y": round(p.coords[0][1], 2)}
        ).to_dict()

    edges = edges[["length", "node_start", "node_end", "geometry"]].to_crs(city_crs)
    edges["type"] = graph_type
    edges["geometry"] = edges["geometry"].apply(
        lambda x: LineString([tuple(round(c, 2) for c in n) for n in x.coords] if x else None)
        )

    travel_type = "walk" if graph_type == "walk" else "car"
    if not speed:
        speed =  4 * 1000 / 60 if graph_type == "walk" else  17 * 1000 / 60

    G = nx.MultiDiGraph()
    for i, edge in tqdm(edges.iterrows(), total=len(edges)):
        p1 = int(edge.node_start)
        p2 = int(edge.node_end)
        geometry = LineString(
            ([(nodes_coord[p1]["x"], nodes_coord[p1]["y"]), (nodes_coord[p2]["x"], nodes_coord[p2]["y"])])
            ) if not edge.geometry else edge.geometry
        G.add_edge(
            p1, p2, length_meter=edge.length, geometry=str(geometry), type = travel_type,
            time_min = round(edge.length / speed, 2)
            )
    nx.set_node_attributes(G, nodes_coord)
    G.graph['crs'] = 'epsg:' + str(city_crs)
    G.graph['graph_type'] = travel_type + " graph"
    G.graph[travel_type + ' speed'] = round(speed, 2)

    print(f"{graph_type.capitalize()} graph done!")
    return G

def public_routes_to_edges(city_osm_id, city_crs, transport_type, speed, boundary):
    routes = overpass_query(get_routes, city_osm_id, transport_type)
    print(f"Extracting and preparing {transport_type} routes:")

    try:
        df_routes = routes.progress_apply(
            lambda x: parse_overpass_route_response(x, city_crs, boundary), axis = 1, result_type="expand"
            )
        df_routes = gpd.GeoDataFrame(df_routes).dropna(subset=["way"]).set_geometry("way")

    except KeyError:
        print(f"It seems there are no {transport_type} routes in the city. This transport type will be skipped.")
        return []

    # some stops don't lie on lines, therefore it's needed to project them
    stop_points = df_routes.apply(lambda x: project_platforms(x, city_crs), axis = 1)

    edges = []
    time_on_stop = 1
    for i, route in stop_points.iterrows():
        length = np.diff(list(route["distance"]))
        for j in range(len(route["pathes"])):
            edge_length = float(length[j])
            p1 = route["pathes"][j][0]
            p2 = route["pathes"][j][1]
            d = {"time_min": round(edge_length/speed + time_on_stop, 2), "length_meter": round(edge_length, 2),
                "type": transport_type, "desc": f"route {i}", "geometry": str(LineString([p1, p2]))}
            edges.append((p1, p2, d))

    return edges

def graphs_spatial_union(G_base, G_to_project):
    points = gpd.GeoDataFrame([[n, Point((d["x"], d["y"]))] for n, d in G_to_project.nodes(data=True)],
                            columns=["node_id_to_project", "geometry"])
    edges_geom = get_nearest_edge_geometry(points, G_base)
    projected_point_info = project_point_on_edge(edges_geom)
    check_point_on_line = projected_point_info.apply(
        lambda x: x.edge_geometry.buffer(1).contains(x.nearest_point_geometry), axis=1).all()
    if not check_point_on_line:
        raise ValueError("Some projected points don't lie on edges")
    points_on_lines = projected_point_info[(projected_point_info["len_from_start"] != 0)
                                            & (projected_point_info["len_to_end"] != 0)]

    points_on_points = projected_point_info[~projected_point_info.index.isin(points_on_lines.index)]
    try:
        points_on_points["connecting_node_id"] = points_on_points.apply(
            lambda x: x.edge_id[0] if x.len_from_start == 0 else x.edge_id[1], axis=1
        )
    except ValueError:
        print("No matching nodes were detected, seems like your data is not the same as in OSM. ")

    updated_G_base, points_on_lines = update_edges(points_on_lines, G_base)
    points_df = pd.concat([points_on_lines, points_on_points])
    united_graph = join_graph(updated_G_base, G_to_project, points_df)
    return united_graph

def get_intermodal_graph(city_osm_id, city_crs, gdf_files, public_transport_speeds=None, walk_speed=None,
                         drive_speed=None):
    G_public_transport: nx.MultiDiGraph = get_public_trasport_graph(city_osm_id, city_crs, gdf_files,
                                                                    public_transport_speeds)
    G_walk: nx.MultiDiGraph = get_osmnx_graph(city_osm_id, city_crs, "walk", speed=walk_speed)

    G_drive: nx.MultiDiGraph = get_osmnx_graph(city_osm_id, city_crs, "drive", speed=drive_speed)
    print("Union of graphs...")
    G_intermodal = graphs_spatial_union(G_walk, G_drive)
    if G_public_transport.number_of_edges() > 0:
        G_intermodal = graphs_spatial_union(G_intermodal, G_public_transport)

    for u, v, d in G_intermodal.edges(data=True):
        if "time_min" not in d:
            d["time_min"] = round(d["length_meter"] / G_walk.graph["walk speed"], 2)
        if "desc" not in d:
            d["desc"] = ""

    for u, d in G_intermodal.nodes(data=True):
        if "stop" not in d:
            d["stop"] = "False"
        if "desc" not in d:
            d["desc"] = ""

    G_intermodal.graph["graph_type"] = "intermodal graph"
    G_intermodal.graph["car speed"] = G_drive.graph["car speed"]
    G_intermodal.graph.update({k: v for k, v in G_public_transport.graph.items() if "speed" in k})
    G_intermodal.graph["created by"] = "CityGeoTools"

    print("Intermodal graph done!")
    return G_intermodal



def get_public_trasport_graph(city_osm_id, city_crs, gdf_files, transport_types_speed=None):
    G = nx.MultiDiGraph()
    edegs_different_types = []
    print("\n")
    if not transport_types_speed:
        transport_types_speed = {
            "subway": 12 * 1000 / 60,
            "tram": 15 * 1000 / 60,
            "trolleybus": 12 * 1000 / 60,
            "bus": 17 * 1000 / 60
        }
    from_file = False
    for transport in gdf_files.values():
        if transport.get('stops') or transport.get('routes'):
            from_file = True

    if not from_file:
        print("Files with routes or with stops was not found. The graph will be built based on data from OSM")
        boundary = overpass_query(get_boundary, city_osm_id)
        boundary = osm2geojson.json2geojson(boundary)
        boundary = geometry.shape(boundary['features'][0]["geometry"])

        for transport_type, speed in transport_types_speed.items():
            print("Getting public routes data from OSM...")
            edges = public_routes_to_edges(city_osm_id, city_crs, transport_type, speed, boundary)
            edegs_different_types.extend(edges)
    else:
        print("Getting public routes data from File...")
        for transport_type, speed in transport_types_speed.items():
            files = gdf_files.get(transport_type)
            if not files.get("routes") or not files.get("stops"):
                print(f"No data provided for \"{transport_type}\", skipping this transport type")
                continue
            else:
                edges = public_routes_to_edges_from_file(city_crs, transport_type, speed, files)
                edegs_different_types.extend(edges)

    G.add_edges_from(edegs_different_types)
    if len(edegs_different_types)==0:
        print(f"No data found for public transport, this graph will be empty.\n")
        return G

    node_attributes = {node: {
        "x": round(node[0], 2), "y": round(node[1], 2), "stop": "True", "desc": []
    } for node in list(G.nodes)}

    for p1, p2, data in list(G.edges(data=True)):
        transport_type = data["type"]
        node_attributes[p1]["desc"].append(transport_type), node_attributes[p2]["desc"].append(transport_type)

    for data in node_attributes.values():
        data["desc"] = ", ".join(set(data["desc"]))
    nx.set_node_attributes(G, node_attributes)
    G = nx.convert_node_labels_to_integers(G)
    G.graph['crs'] = 'epsg:' + str(city_crs)
    G.graph['graph_type'] = "public transport graph"
    G.graph.update({k + " speed": round(v, 2) for k, v in transport_types_speed.items()})

    print("Public transport graph done!")
    return G

def public_routes_to_edges_from_file(city_crs, transport_type, speed, gdf_files):
    edges = []
    try:
        gdf_stops = gpd.read_file(gdf_files.get("stops"))

        gdf_routes = gpd.read_file(gdf_files.get("routes"))
        ways: gpd.GeoDataFrame = gdf_routes[['route', 'geometry']].copy()
        ways = ways.explode(index_parts=False).to_crs(city_crs)
        ways["route"] = ways["route"].apply(lambda x: str(x).strip())
        ways.set_index('route', inplace=True)

        platforms = pd.DataFrame(columns=["route", "geometry"])

        for index, row in gdf_stops.iterrows():
            for i in str(row["route"]).split(","):
                df = pd.DataFrame(({i: row["geometry"]}).items(), columns=['route', 'geometry'])
                platforms = pd.concat([platforms, df])
        platforms.reset_index(inplace=True, drop=True)
        platforms["route"] = platforms["route"].apply(lambda x: str(x).strip())
        df_routes = pd.DataFrame()

        for index, row in ways.iterrows():
            platforms_: pd.Series = platforms[platforms["route"] == str(index)].drop('route', axis=1).reset_index(
                drop=True).squeeze()
            series = pd.Series({'way': row["geometry"], 'platforms': platforms_})
            df_routes = pd.concat([df_routes, series], axis=1, ignore_index=True)
        df_routes = df_routes.transpose()
        df_routes = gpd.GeoDataFrame(df_routes).set_geometry("way")

        stop_points = df_routes.apply(lambda x: project_platforms(x, city_crs), axis=1)

        time_on_stop = 1
        for i, route in stop_points.iterrows():
            length = np.diff(list(route["distance"]))
            for j in range(len(route["pathes"])):
                edge_length = float(length[j])
                p1 = route["pathes"][j][0]
                p2 = route["pathes"][j][1]
                d = {"time_min": round(edge_length / speed + time_on_stop, 2), "length_meter": round(edge_length, 2),
                     "type": transport_type, "desc": f"route {i}", "geometry": str(LineString([p1, p2]))}
                edges.append((p1, p2, d))
    except KeyError:
        print(f"! ! !\nThe 'route' column was not found in one of the files for \"{transport_type}\" . Please check their contents.\n! ! !")
    except Exception as err:
        print(f"! ! !\nFile with routes or with stops was not found for \"{transport_type}\", error:",err,"\n! ! !")
    finally:
        return edges

In [ ]:
city_osm_id = 7656650
city_crs = 32643
"""
If you need to upload public transportation data from a file, please fill in the dictionary below.
Leave 'None' in case of no file or provide the file name with the .geojson format.
For example:
"bus": {"stops": "bus_stop_Tara.geojson", "routes": "Tara_routes.geojson"}
OR
"bus": {"stops": None, "routes": None}"
"""
gdf_files = {
    "subway": {"stops": None, "routes": None},

    "tram": {"stops": None, "routes": None},

    "trolleybus": {"stops": None, "routes": None},

    "bus": {"stops": "bus_stop_Tara.geojson", "routes": "Тара_маршруты.geojson"}
}

G_graph: nx.MultiDiGraph = get_intermodal_graph(city_osm_id, city_crs, gdf_files)
nx.write_graphml(G_graph, f'{city_osm_id}.graphml')
for i in G_graph.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
ec = ['y' if 'walk' in d['type'] else 'r' if 'car' in d['type'] else 'b' for _, _, _, d in
      G_graph.edges(keys=True, data=True)]
ox.plot_graph(G_graph, edge_color=ec, dpi=300, save=True, filepath=f"{city_osm_id}.png", node_size=2)